# Investigating GMDI scores calculated using LOO and non-LOO, but with the base linear features (instead of any stump features)

In [125]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import r2_score, mean_squared_error
from tqdm import tqdm

from imodels.importance.representation_cleaned import IdentityTransformer
from imodels.importance.r2f_exp_cleaned import GenericLOOPPM, GenericPPM, GMDI, RidgePPM, RidgeLOOPPM

In [128]:
def one_iter(n, p, beta, eps, mode="keep_k"):
    X = np.random.randn(n, p)
    y = X @ beta + np.random.randn(n) * eps
    loo_ppm = GenericLOOPPM(estimator=LinearRegression(), alpha_grid=[0])
    ppm = GenericPPM(estimator=LinearRegression())
    gmdi_loo = GMDI(IdentityTransformer(n_features=p), loo_ppm, mean_squared_error, mode=mode)
    gmdi = GMDI(IdentityTransformer(n_features=p), ppm, mean_squared_error, mode=mode)
    loo_scores = gmdi_loo.get_scores(X, y)
    nonloo_scores = gmdi.get_scores(X, y)

    X = X - X.mean(axis=0)
    X = np.hstack([np.ones((X.shape[0], 1)), X])
    M = X.T @ X
    M_inv = np.linalg.inv(M)
    P = np.diag([1, 1] + [0] * (p-1))
    I = np.eye(n)
    H = X @ P @ M_inv @ X.T
    x_only_score = np.trace((I - H) @ (I - H).T) * (eps**2) / n + 2

    return loo_scores, nonloo_scores, x_only_score

def one_iter_ridge(n, p, beta, eps, mode="keep_k"):
    X = np.random.randn(n, p)
    y = X @ beta + np.random.randn(n) * eps
    loo_ppm = RidgeLOOPPM()
    ppm = RidgePPM(alphas=np.logspace(-5, 5, 100))
    gmdi_loo = GMDI(IdentityTransformer(n_features=p), loo_ppm, mean_squared_error, mode=mode)
    gmdi = GMDI(IdentityTransformer(n_features=p), ppm, mean_squared_error, mode=mode)
    loo_scores = gmdi_loo.get_scores(X, y)
    nonloo_scores = gmdi.get_scores(X, y)
    return loo_scores, nonloo_scores

def full_scores_one_iter(n, p, beta, eps):
    X = np.random.randn(n, p)
    y = X @ beta + np.random.randn(n) * eps
    loo_ppm = GenericLOOPPM(estimator=LinearRegression(), alpha_grid=[0])
    ppm = GenericPPM(estimator=LinearRegression())
    gmdi_loo = GMDI(IdentityTransformer(n_features=p), loo_ppm, mean_squared_error, mode="keep_rest")
    gmdi = GMDI(IdentityTransformer(n_features=p), ppm, mean_squared_error, mode="keep_rest")
    gmdi_loo.get_scores(X, y)
    gmdi_loo_full_scores = gmdi_loo.scoring_fn(y, gmdi_loo.partial_prediction_model.get_full_predictions())
    gmdi.get_scores(X, y)
    gmdi_full_scores = gmdi.scoring_fn(y, gmdi.partial_prediction_model.get_full_predictions())

    return gmdi_loo_full_scores, gmdi_full_scores

def linear_reg_partial_pred(n, p, beta, eps):
    X = np.random.randn(n, p)
    y = X @ beta + np.random.randn(n) * eps
    scores = []
    for i in range(n):
        train_indices = [j != i for j in range(n)]
        X_partial = X[train_indices, :]
        y_partial = y[train_indices]
        lr = LinearRegression()
        lr.fit(X_partial, y_partial)
        coef = lr.coef_[0]
        partial_pred = coef * X[i, 0]
        score = (partial_pred - y[i]) ** 2
        scores.append(score)
    return np.mean(scores)


In [132]:
def _get_loo_fitted_parameters(X, y, coef_):
    orig_preds = X @ coef_
    J = X.T @ X
    normal_eqn_mat = np.linalg.inv(J) @ X.T
    h_vals = np.sum(X.T * normal_eqn_mat, axis=0)
    loo_fitted_parameters = coef_[:, np.newaxis] + normal_eqn_mat * (orig_preds - y) / (1 - h_vals)
    return loo_fitted_parameters

In [147]:
def compare_fitted_coefs(n, p, beta, eps):
    X = np.random.randn(n, p)
    y = X @ beta + np.random.randn(n) * eps
    coefs = []

    for i in range(n):
        train_indices = [j != i for j in range(n)]
        X_partial = X[train_indices, :]
        y_partial = y[train_indices]
        lr = LinearRegression()
        lr.fit(X_partial, y_partial)
        coefs.append(lr.coef_)
    X_aug = np.hstack([X, np.ones((X.shape[0], 1))])
    full_lr = LinearRegression()
    full_lr.fit(X, y)
    coef_aug = np.array(list(full_lr.coef_) + [full_lr.intercept_])
    alt_coefs = _get_loo_fitted_parameters(X_aug, y, coef_aug)

    return np.array(coefs), alt_coefs

## Results using linear regression

In [121]:
n = 20
p = 10
q = 3
beta = np.array([1] * q + [0] * (p - q))
eps = 1
n_iter = 1000

In [122]:
all_loo_scores = []
all_nonloo_scores = []
x_only_scores = []
for i in tqdm(range(5000)):
    loo_score, nonloo_score, x_only_score = one_iter(n, p, beta, eps, mode="keep_k")
    all_loo_scores.append(loo_score)
    all_nonloo_scores.append(nonloo_score)
    x_only_scores.append(x_only_score)

100%|██████████| 5000/5000 [00:45<00:00, 110.90it/s]


In [123]:
loo_x1 = np.array(all_loo_scores)[:,0]
nonloo_x1 = np.array(all_nonloo_scores)[:,0]

In [124]:
np.mean(loo_x1), np.std(loo_x1), np.mean(nonloo_x1), np.std(nonloo_x1), np.mean(x_only_scores), np.std(x_only_scores)

(3.1238364383412973,
 0.9932067612610507,
 2.875062052576607,
 0.9482758676057708,
 2.9553844274511394,
 0.045730854072444385)

True value = 3, can see that both LOO and non-LOO values are off by 0.12, but in opposite directions.

The third mean value is an estimate of the expected non-LOO value calculated using theory to remove some of the randomness. It is off by a little from the fully empirical non-LOO value and I'm not sure yet why at the moment.

## Results using ridge regression

In [129]:
all_loo_scores = []
all_nonloo_scores = []
for i in tqdm(range(5000)):
    loo_score, nonloo_score = one_iter_ridge(n, p, beta, eps, mode="keep_k")
    all_loo_scores.append(loo_score)
    all_nonloo_scores.append(nonloo_score)

100%|██████████| 5000/5000 [02:23<00:00, 34.82it/s]


In [132]:
loo_x1 = np.array(all_loo_scores)[:,0]
nonloo_x1 = np.array(all_nonloo_scores)[:,0]

In [133]:
np.mean(loo_x1), np.std(loo_x1), np.mean(nonloo_x1), np.std(nonloo_x1)

(3.1234855295273833,
 0.9701086850873901,
 2.8493847263311354,
 0.9174822806138582)

True value = 3. This time LOO is off by 0.12, non-LOO is off by 0.15.